In [1]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
def extract_hog_features(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Compute HOG features
    features = hog(gray, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2))
    return features

In [4]:
dataset_dir = "out"

# List to store image paths and corresponding labels
image_paths = []
labels = []
hog_features=[]
from tqdm import tqdm
# Iterate through each font type directory
for font_type in os.listdir(dataset_dir):
    counter=0;
    font_type_dir = os.path.join(dataset_dir, font_type)
    if os.path.isdir(font_type_dir):
        # Iterate through each image in the font type directory
        for image_file in tqdm (os.listdir(font_type_dir)):
            counter+=1;
            if counter>1000:
                break;
            image_path = os.path.join(font_type_dir, image_file)
            # Read the image
            image = cv2.imread(image_path)
            if image is not None:
                # Extract HOG features
                hog_features.append (extract_hog_features(image))
                # Add HOG features to the list
                image_paths.append(image_path)
                labels.append(font_type)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [02:00<00:00,  8.33it/s]


In [5]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(hog_features, labels, test_size=0.2, random_state=42)


In [6]:
# Convert lists to numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)

In [7]:
# Train SVM model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

SVC(kernel='linear')

In [8]:

# Predict on the test set
y_pred = svm_model.predict(X_test)

In [9]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.69


In [10]:
import joblib

# Save the trained SVM model to a file
joblib.dump(svm_model, 'svm_model_linear.pkl')


#loaded_model = joblib.load('svm_model.pkl')

['svm_model_linear.pkl']

In [ ]:
#rbf
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train, y_train)

In [ ]:
y_pred = svm_model.predict(X_test)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)